In [1]:
import ee
import geemap

import os

# --- ！！！网络代理设置！！！ ---
# 在这里设置您的HTTP和HTTPS代理。
# 这个设置只会影响当前这个Python脚本的运行。
# 请将 '127.0.0.1:7890' 替换为您自己的代理地址和端口。
proxy_url = 'http://127.0.0.1:7897' 
os.environ['http_proxy'] = proxy_url
os.environ['https_proxy'] = proxy_url
# 不再需要 import sys, import os (除非其他地方用到)
# 也不再需要任何 sys.path.append(...)

# 初始化GEE
ee.Initialize(project='geemap-441216') # 替换为您的项目ID

# Python会自动在当前文件夹下寻找模块，所以可以直接导入！
try:
    import z_flood_robust
    from z_flood_robust import zscore # 甚至可以这样导入
    zscore.calc_basemad # 尝试访问
    print("本地自定义模块 z_flood_robust 已成功导入！")
except ImportError as e:
    print(f"导入模块失败: {e}")
    print("请确保'z_flood_robust'文件夹与您的Jupyter笔记本在同一个目录下。")

本地自定义模块 z_flood_robust 已成功导入！


# 1.导入依赖-数据管理模块

In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle #用于在图表中添加形状(例如矩形)
import re #正则表达式模块
from z_flood_robust import calc_basemedian, calc_basemad, calc_median_anomaly, calc_robust_zscore, calc_basemean,calc_basestd,calc_zscore
from z_flood_robust import mapFloods,floodPalette

from ipywidgets import Label #用于创建交互式控件



# 2.定义交互式地图以选择感兴趣区域-界面与交互模块

In [4]:
#解析用户点击地图时生成的坐标字符串，提取经纬度并返回
def parseClickedCoordinates(label):
  #利用正则表达式，从label.value提取出坐标：一个可能带有负号的浮点数 这里为【经度，纬度】
  #正则表达式： r'(?:-)?[0-9]+.[0-9]+'
  #r'表示原始字符，(?:-)是非捕获组，?:表示负号是可选的，[0-9]+表示匹配一个数字，+表示可以出现一次或多次，.匹配小数点，最后再次匹配一个或多个数字
  coords = [float(c) for c in re.findall(r'(?:-)?[0-9]+.[0-9]+', label.value)]
  coords.reverse() #反转为【纬度，经度】，符合GEE坐标格式
  return coords

#创建一个Lable，用于显示用户点击的目标
l = Label()
display(1)
#处理用户与地图的交互事件，当用户点击地图时，将点击的坐标存储到Label控件中
def handle_interaction(**kwargs):
  #kwargs包含交互事件的参数 kwargs.get('type'):获取事件类型为鼠标点击 kwargs.get('coordinates')：获取点击的坐标，转换为字符串并存储到Label控件中
  if kwargs.get('type') == 'click':
    l.value = str(kwargs.get('coordinates'))

print('请点击地图以选择你要监测的区域')
#创建交互式地图
Map = geemap.Map()
Map.on_interaction(handle_interaction)
Map

1

请点击地图以选择你要监测的区域


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

# 3.定义几何范围并展示-界面与交互模块

## 3.1.手动划定研究区域

In [13]:
lon,lat = parseClickedCoordinates(l)
w,h = 2,2 #矩形宽度与高度（单位：度）

geometry = ee.Geometry.Polygon(
    [[[lon-w,lat-h],
     [lon-w,lat+h],
     [lon+w,lat+h],
     [lon+w,lat-h]]]
)

#将几何范围添加到地图
Map.addLayer(
    geometry,
    {'color':'red','fillColor':'00000000'},
    'AOI'
)
Map

Map(bottom=399611.0, center=[39.38139089214383, -0.3865922574188186], controls=(WidgetControl(options=['positi…

In [14]:
#USE MAP RECTANGLE 选择区域
#获取感兴趣区域（用于论文绘图)
roi_choose1 = Map.user_roi

if roi_choose1 is not None:
  #获取ROI类型
  roi_type = roi_choose1.type().getInfo()
  print(f"ROI 类型：{roi_type}")

  #如果是Polygon，获取坐标
  if roi_type == 'Polygon':
    coords = roi_choose1.coordinates().getInfo()
    print(f"ROI 坐标：{coords}")

ROI 类型：Polygon
ROI 坐标：[[[-0.424067, 39.459907], [-0.424067, 39.468588], [-0.414195, 39.468588], [-0.414195, 39.459907], [-0.424067, 39.459907]]]


## 3.2.从GEE资产里加载研究区

In [3]:
asset_id = 'projects/geemap-441216/assets/roi_sci_valencia_small'

try:
    # 从GEE Assets加载矢量文件作为FeatureCollection
    aoi_fc = ee.FeatureCollection(asset_id)
    
    
    # 对于洪水分析，我们通常需要的是所有要素合并后的几何边界
    # .geometry() 会将FeatureCollection中所有的要素融合成一个单一的几何对象
    roi_choose1 = aoi_fc.geometry()
    
    print("成功从GEE Assets加载研究区！")
    
    # 创建一个地图来可视化你的AOI，确保加载正确
    Map = geemap.Map()
    Map.addLayer(roi_choose1, {'color': 'red', 'fillColor': '00000000'}, 'Study Area (from Asset)')
    Map.centerObject(roi_choose1, 10) # 自动缩放到研究区
    display(Map)
    
except ee.EEException as e:
    print(f"加载AOI失败！请检查你的Asset ID是否正确: {asset_id}")
    print(f"错误信息: {e}")
    # 如果加载失败，停止执行后续代码
    raise SystemExit("无法加载AOI，程序终止。")
    

成功从GEE Assets加载研究区！


Map(center=[39.39610997738956, -0.377311304937758], controls=(WidgetControl(options=['position', 'transparent_…

# 4 设置日期，过滤影像计算数量（以下是西班牙巴伦西亚区域）

In [4]:
targdate = '2024-10-31'   # 目标日期 (需要根据实际数据可用性验证或微调)
basestart = '2023-11-20'  # 基线开始日期
baseend = '2024-09-10'   # 基线结束日期

# !!! (可选) 替换硬编码的 aoi2 !!!
# 如果你决定使用固定的 valencia_aoi，在这里替换：
#aoi3 = ee.Geometry.Rectangle([-0.55, 39.35, -0.30, 39.50]) # 新的巴伦西亚AOI
# !!! 更好的做法是完全不用 aoi2，下面 filterBounds 直接用 analysis_aoi !!!

filters = [
    ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
    ee.Filter.listContains("transmitterReceiverPolarisation", "VH"), # 如果需要VH也取消注释
    ee.Filter.equals("instrumentMode", "IW"),
    # !!! 重要：检查轨道方向 !!!
    # 需要确认 targdate 前后过境巴伦西亚的 S1 影像是升轨还是降轨
    # 可能需要改为 'DESCENDING'，或不加这个过滤器以获取所有轨道
    ee.Filter.equals("orbitProperties_pass", "ASCENDING"), # <--- 可能需要修改或移除!
    # 时间过滤器现在是动态的，覆盖基线期和目标期，无需修改
    ee.Filter.date('2022-08-21', ee.Date(targdate).advance(1, 'day'))
]

# 加载S1数据并计算Z分数
s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filter(filters) \
    .filterBounds(roi_choose1) # !!! 使用统一的 analysis_aoi !!!



# 检查是否有影像满足条件
s1_size = s1.size().getInfo()
print(f"找到满足条件的 Sentinel-1 影像数量: {s1_size}")
if s1_size == 0:
    print("警告：未找到满足条件的影像，请检查时间范围、AOI 或轨道方向过滤器！")
print(s1.aggregate_array('system:time_start').map(lambda t: ee.Date(t).format()).getInfo())

# Z 分数计算 (轨道方向参数需要与上面 filter 保持一致)
# orbit_direction = "ASCENDING" # <--- 如果上面修改了这里也要改
orbit_direction = "ASCENDING" if ee.Filter.equals("orbitProperties_pass", "ASCENDING") in filters else "DESCENDING" #或者更灵活的方式
z = calc_zscore(s1, basestart, baseend, 'IW', orbit_direction)
rz = calc_robust_zscore(s1, basestart, baseend, 'IW', orbit_direction)


MAD = calc_basemad(s1, basestart, baseend, 'IW', orbit_direction)

找到满足条件的 Sentinel-1 影像数量: 133
['2022-08-26T17:54:58', '2022-08-31T18:02:53', '2022-09-07T17:54:59', '2022-09-12T18:02:54', '2022-09-19T17:54:59', '2022-09-24T18:02:54', '2022-10-01T17:54:59', '2022-10-06T18:02:54', '2022-10-13T17:54:59', '2022-10-25T17:55:00', '2022-10-30T18:02:54', '2022-11-06T17:54:59', '2022-11-11T18:02:54', '2022-11-18T17:54:59', '2022-11-23T18:02:54', '2022-11-30T17:54:58', '2022-12-05T18:02:53', '2022-12-12T17:54:58', '2022-12-17T18:02:52', '2022-12-24T17:54:57', '2022-12-29T18:02:52', '2023-01-05T17:54:57', '2023-01-10T18:02:51', '2023-01-17T17:54:56', '2023-01-22T18:02:51', '2023-01-29T17:54:56', '2023-02-03T18:02:51', '2023-02-10T17:54:55', '2023-02-15T18:02:50', '2023-02-22T17:54:55', '2023-02-27T18:02:50', '2023-03-06T17:54:55', '2023-03-11T18:02:50', '2023-03-18T17:54:55', '2023-03-23T18:02:50', '2023-03-30T17:54:55', '2023-04-04T18:02:50', '2023-04-11T17:54:56', '2023-04-16T18:02:51', '2023-04-23T17:54:56', '2023-04-28T18:02:51', '2023-05-05T17:54:57', '202

# 5.对照方法

## 5.1 全局OTSU阈值

In [5]:
# --- 1. 定义您提供的Otsu函数 ---
# (这是您在原始代码中编写的、在GEE服务器端运行的Otsu算法)
def otsu(histogram):
  histogram = ee.Dictionary(histogram);
  counts = ee.Array(histogram.get('histogram'))
  means = ee.Array(histogram.get('bucketMeans'))
  size = means.length().get([0])
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
  sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
  mean = sum.divide(total)
  
  indices = ee.List.sequence(1, size)
  
  def compute_bss(i):
    aCounts = counts.slice(0, 0, i)
    aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCount)
    bCount = total.subtract(aCount)
    bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
    return aCount.multiply(aMean.subtract(mean).pow(2)).add(bCount.multiply(bMean.subtract(mean).pow(2)))

  bss = indices.map(compute_bss)
  
  return means.sort(bss).get([-1])

In [6]:
# --- 2. 获取洪水目标日的SAR影像 ---
target_date_ee = ee.Date(targdate)
s1_target_collection = s1.filterDate(target_date_ee, target_date_ee.advance(1, 'day'))

target_image_size = s1_target_collection.size().getInfo()
if target_image_size == 0:
    print(f"错误：在目标日期 {targdate} 未找到任何Sentinel-1影像，无法执行Otsu法。")
else:
    print(f"在 {targdate} 找到 {target_image_size} 景影像，将合并后进行处理。")
    kernel_radius = 2
    sar_image_for_otsu = s1_target_collection.select('VV').mosaic().clip(roi_choose1)\
                                             .focal_mean(radius = kernel_radius, units='pixels')

    # --- 3. 加载JRC永久水体数据并创建掩膜 ---
    # print("加载JRC全球地表水数据...")
    # jrc_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
    
    # 核心修正
    # 使用.unmask(0)将所有无数据的像元(如陆地)赋值为0
    # occurrence_unmasked = jrc_water.select('occurrence').unmask(0)
    
    # permanent_water_mask = occurrence_unmasked.gt(90)
    # print("永久性水体掩膜创建完成 (occurrence > 90%)。")

    # --- 4. 计算Otsu阈值并提取初始水体 ---
    print("正在计算影像直方图...")
    histogram_dict = sar_image_for_otsu.reduceRegion(
        reducer=ee.Reducer.histogram(255, 0.1), # buckets, maxBuckets
        geometry=roi_choose1,
        scale=30,
        maxPixels=1e10
    )
    
    # !!! 核心修正：调用您自己定义的 otsu 函数 !!!
    otsu_threshold = otsu(histogram_dict.get('VV'))
    
    otsu_threshold_value = otsu_threshold.getInfo()
    print(f"计算出的全局Otsu阈值为: {otsu_threshold_value:.4f} dB")
    
    # 进行亮暗目标分割（不移除永久水体）
    dark_targets_otsu = sar_image_for_otsu.lt(otsu_threshold_value)
    bright_targets_otsu = sar_image_for_otsu.gte(otsu_threshold_value)

    # --- 5. 移除永久性水体，得到最终洪水范围 ---
    # flood_map_otsu = initial_water_otsu.And(permanent_water_mask.Not())
    # print("已从Otsu结果中移除永久性水体，生成最终洪水图。")

    # --- 6. 可视化结果 ---
    print("正在准备可视化地图...")
    Map_Otsu = geemap.Map()
    Map_Otsu.centerObject(roi_choose1, 11)

    sar_vis_params = {'min': -25, 'max': 5}
    dark_vis_params = {'palette': ['#0000FF']}
    #bright_vis_params = {'palette': ['#FF0000']}

    Map_Otsu.addLayer(sar_image_for_otsu, sar_vis_params, f'SAR Image ({targdate})')
    # Map_Otsu.addLayer(permanent_water_mask.selfMask(), {'palette': ['#CCCCCC']}, 'JRC Permanent Water (>90%)')
    Map_Otsu.addLayer(dark_targets_otsu.selfMask(), dark_vis_params, 'Dark Targets (Otsu)')
    # Map_Otsu.addLayer(bright_targets_otsu.selfMask(), bright_vis_params, 'Bright Targets (Otsu)')
    
    display(Map_Otsu)

在 2024-10-31 找到 1 景影像，将合并后进行处理。
正在计算影像直方图...
计算出的全局Otsu阈值为: -13.5002 dB
正在准备可视化地图...


Map(center=[39.39610997738956, -0.377311304937758], controls=(WidgetControl(options=['position', 'transparent_…

## 5.2 采用K-I方法

### 5.2.1 KI方法函数

K-I方法理论基础：假设图像的直方图由两个高斯分布组成（前景和背景），通过最小化分类误差来找到最优阈值
目标函数（需要最小化）：J(T) = 1 + 2[P₀log(σ₀) + P₁log(σ₁)] - 2[P₀log(P₀) + P₁log(P₁)]
其中：
T：阈值
P₀, P₁：两类的先验概率
σ₀, σ₁：两类的标准差

In [26]:
# 完全替换 ### 5.2.1 KI方法函数 单元格中的代码

def kittler_illingworth(histogram):
    """
    Kittler-Illingworth最小误差阈值分割方法
    
    理论基础：
    1. 假设图像直方图由两个高斯分布组成（暗区域和亮区域）
    2. 通过最小化贝叶斯分类误差来确定最优阈值
    3. 考虑每个类别的均值、方差和先验概率
    
    Args:
        histogram: GEE直方图字典，包含'histogram'(频数)和'bucketMeans'(区间中心值)
    
    Returns:
        最优阈值（使分类误差最小的阈值）
    """
    # 将输入转换为GEE字典
    histogram = ee.Dictionary(histogram)
    
    # 提取直方图基本信息
    counts = ee.Array(histogram.get('histogram'))      # 每个区间的像素数量（频数）
    means = ee.Array(histogram.get('bucketMeans'))     # 每个区间的中心（SAR值）
    size = means.length().get([0])                     # 区间数量
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])  # 总像素数
    
    # 创建阈值候选索引序列（避免边界问题，从1到size-2）
    indices = ee.List.sequence(1, size.subtract(2))
    
    def compute_ki_criterion(i):
        """
        计算特定阈值位置i的K-I目标函数值

        Args:
            i: 阈值在直方图中的索引位置
        Returns:
            目标函数值（需要最小化）
        """
        i = ee.Number(i)  # 确保i是GEE数值类型
        
        # === 计算暗区域（类别0：索引0到i-1）的统计参数 ===
        
        # 提取暗区的频数和均值
        class0_counts = counts.slice(0, 0, i)           # 取前i个区间的频数
        class0_means = means.slice(0, 0, i)             # 取前i个区间的均值
        
        # 计算暗区域的总像素数
        n0 = class0_counts.reduce(ee.Reducer.sum(), [0]).get([0])
        # 防止除零错误：如果n0为0，设为1
        n0 = ee.Number(ee.Algorithms.If(n0.eq(0), 1, n0))
        
        # 计算暗区域的加权平均值（均值）
        # μ₀ = Σ(h_i × x_i) / Σ(h_i)，其中h_i是频数，x_i是区间中心值
        weighted_sum0 = class0_means.multiply(class0_counts)\
                                   .reduce(ee.Reducer.sum(), [0]).get([0])
        mu0 = weighted_sum0.divide(n0)
        
        # 计算暗区域的方差
        # σ₀² = Σ[h_i × (x_i - μ₀)²] / Σ(h_i)
        diff0_squared = class0_means.subtract(mu0).pow(2)  # (x_i - μ₀)²
        variance_sum0 = diff0_squared.multiply(class0_counts)\
                                   .reduce(ee.Reducer.sum(), [0]).get([0])
        var0 = variance_sum0.divide(n0)
        # 防止方差为0（会导致log(0)错误）
        var0 = ee.Number(ee.Algorithms.If(var0.eq(0), 0.001, var0))
        
        # === 计算亮区域（类别1：索引i到size-1）的统计参数 ===
        
        # 提取亮区域的频数和均值
        class1_counts = counts.slice(0, i, size)
        class1_means = means.slice(0, i, size)
        
        # 计算亮区域的总像素数
        n1 = class1_counts.reduce(ee.Reducer.sum(), [0]).get([0])
        # 防止除零错误
        n1 = ee.Number(ee.Algorithms.If(n1.eq(0), 1, n1))
        
        # 计算亮区域的加权平均值
        weighted_sum1 = class1_means.multiply(class1_counts)\
                                   .reduce(ee.Reducer.sum(), [0]).get([0])
        mu1 = weighted_sum1.divide(n1)
        
        # 计算亮区域的方差
        diff1_squared = class1_means.subtract(mu1).pow(2)
        variance_sum1 = diff1_squared.multiply(class1_counts)\
                                   .reduce(ee.Reducer.sum(), [0]).get([0])
        var1 = variance_sum1.divide(n1)
        # 防止方差为0
        var1 = ee.Number(ee.Algorithms.If(var1.eq(0), 0.001, var1))
        
        # === 计算先验概率 ===
        # P₀ = n₀/N，P₁ = n₁/N，其中N是总像素数
        p0 = n0.divide(total)  # 暗区域的先验概率
        p1 = n1.divide(total)  # 亮区域的先验概率
        
        # === 计算Kittler-Illingworth准则函数 ===
        # J(T) = 1 + 2[P₀log(σ₀) + P₁log(σ₁)] - 2[P₀log(P₀) + P₁log(P₁)]
        
        # 计算标准差（σ = √variance）
        sigma0 = var0.sqrt()
        sigma1 = var1.sqrt()
        
        # 第一项：常数项
        term1 = ee.Number(1)
        
        # 第二项：2[P₀log(σ₀) + P₁log(σ₁)]
        # 这一项反映了每个类别内部的不确定性
        term2 = p0.multiply(sigma0.log()).add(p1.multiply(sigma1.log())).multiply(2)
        
        # 第三项：-2[P₀log(P₀) + P₁log(P₁)]
        # 这一项是信息熵项，反映了类别间的不确定性
        entropy_term = p0.multiply(p0.log()).add(p1.multiply(p1.log()))
        term3 = entropy_term.multiply(2)
        
        # 最终的准则函数值
        criterion = term1.add(term2).subtract(term3)
        
        return criterion.multiply(-1)
    
    # === 方法1：使用内置的sort操作找到最小值的索引,最小值要转换为找到-1*后的最大值 ===
    criteria_values = indices.map(compute_ki_criterion)
    
    optimal_means_value = means.slice(0,1,size.subtract(1)).sort(criteria_values).get([-1])
    
    return optimal_means_value
    
    
    
    # === 方法2：手动迭代，找到最小值的索引 ===
    # 对所有候选阈值位置计算K-I准则函数值
    # criteria_values = indices.map(compute_ki_criterion)
    
    # # 将准则函数值和索引转换为列表
    # criteria_list = ee.List(criteria_values)
    # indices_list = ee.List(indices)
    
    # # 获取列表长度
    # list_size = criteria_list.size()
    
    # # 使用reduce操作找到最小值的索引
    # def find_min_with_iterate():
    #     """
    #     使用iterate方法找到最小值及其索引
    #     """
    #     # 创建位置索引序列 [0, 1, 2, ...]
    #     position_sequence = ee.List.sequence(0, list_size.subtract(1))
        
    #     def compare_function(current_pos, accumulator):
    #         """
    #         比较函数：比较当前位置的值与累积器中的最小值
            
    #         Args:
    #             current_pos: 当前位置索引（在criteria_list中的位置）
    #             accumulator: [最小值, 对应的原始索引]
    #         """
    #         current_pos = ee.Number(current_pos)
            
    #         # 获取当前位置的准则函数值
    #         current_value = ee.Number(criteria_list.get(current_pos))
    #         # 获取当前位置对应的原始索引
    #         current_original_index = ee.Number(indices_list.get(current_pos))
            
    #         # 从累积器中提取当前最小值信息
    #         acc_list = ee.List(accumulator)
    #         acc_min_value = ee.Number(acc_list.get(0))    # 当前最小值
    #         acc_min_index = ee.Number(acc_list.get(1))    # 对应的原始索引
            
    #         # 如果当前值更小，更新累积器；否则保持不变
    #         return ee.Algorithms.If(
    #             current_value.lt(acc_min_value),
    #             [current_value, current_original_index],  # 更新为当前值
    #             [acc_min_value, acc_min_index]           # 保持原值
    #         )
        
    #     # 初始化：使用第一个元素
    #     first_value = ee.Number(criteria_list.get(0))
    #     first_original_index = ee.Number(indices_list.get(0))
    #     initial_accumulator = [first_value, first_original_index]
        
    #     # 从第二个元素开始迭代（位置1开始）
    #     remaining_positions = position_sequence.slice(1)
        
    #     # 执行迭代比较
    #     final_result = ee.List(remaining_positions.iterate(compare_function, initial_accumulator))
        
    #     # 返回最优的原始索引
    #     return ee.Number(final_result.get(1))
    
    # # 找到最优索引
    # optimal_index = find_min_with_iterate()
    
    # # === 修复：正确从Array中获取值 ===
    # # 问题：Array.get()需要整数列表，但我们传递的是浮点数
    # # 解决：将浮点数转换为整数，并用列表格式传递
    # optimal_index_int = optimal_index.int()  # 转换为整数
    
    # # 从means数组中获取最优阈值对应的SAR值
    # # Array.get()需要一个整数列表作为位置参数
    # return means.get([optimal_index_int])

# 显示函数定义完成
print("✅ Kittler-Illingworth函数定义完成")
print("📖 该方法基于最小分类误差准则，假设图像由两个高斯分布组成")
print("🔧 已修复Array.get()类型转换问题")

✅ Kittler-Illingworth函数定义完成
📖 该方法基于最小分类误差准则，假设图像由两个高斯分布组成
🔧 已修复Array.get()类型转换问题


In [27]:
# 在 ### 5.2.1 KI方法函数 单元格后添加新的代码单元格

### 5.2.2 应用K-I方法进行SAR图像分割

# 检查是否有目标日期的SAR影像可用
if target_image_size > 0:
    print(f"\n🔬 === Kittler-Illingworth 最小误差分割方法 ===")
    print(f"📅 处理日期: {targdate}")
    print(f"🛰️  使用影像数量: {target_image_size} 景")
    
    # === 步骤1: 使用与Otsu相同的预处理SAR影像 ===
    print("\n📡 使用相同的预处理SAR影像...")
    print(f"   - 波段: VV极化")
    print(f"   - 滤波核半径: {kernel_radius} 像素")
    print(f"   - 研究区域: 已裁剪到AOI")
    
    # === 步骤2: 计算图像直方图 ===
    print("\n📊 正在计算SAR影像的直方图...")
    print("   - 这是K-I方法的基础数据")
    print("   - 直方图将用于估计两个高斯分布的参数")
    
    # 使用与Otsu相同的直方图参数确保一致性
    histogram_dict_ki = sar_image_for_otsu.reduceRegion(
        reducer=ee.Reducer.histogram(255, 0.1),  # 255个区间，区间宽度0.1dB
        geometry=roi_choose1,                     # 限制在研究区域内
        scale=30,                                 # 30米空间分辨率
        maxPixels=1e10                           # 最大处理像素数
    )
    
    # === 步骤3: 应用K-I算法计算最优阈值 ===
    print("\n🧮 正在应用Kittler-Illingworth算法...")
    print("   - 假设: SAR图像由暗目标(水体)和亮目标(陆地)两个高斯分布组成")
    print("   - 目标: 找到使贝叶斯分类误差最小的阈值")
    
    # 调用我们定义的K-I函数
    ki_threshold = kittler_illingworth(histogram_dict_ki.get('VV'))
    ki_threshold_value = ki_threshold.getInfo()
    
    print(f"✅ K-I最小误差阈值计算完成: {ki_threshold_value:.4f} dB")
    
    # === 步骤4: 与Otsu方法进行对比分析 ===
    print(f"\n📈 === 阈值对比分析 ===")
    print(f"   🎯 Otsu阈值 (最大类间方差):     {otsu_threshold_value:.4f} dB")
    print(f"   🎯 K-I阈值 (最小分类误差):      {ki_threshold_value:.4f} dB")
    
    threshold_diff = ki_threshold_value - otsu_threshold_value
    print(f"   📏 阈值差异:                    {threshold_diff:+.4f} dB")
    
    # 分析阈值差异的含义
    if abs(threshold_diff) < 0.5:
        print("   💡 两种方法结果相近，图像可能具有明显的双峰特征")
    elif ki_threshold_value < otsu_threshold_value:
        print("   💡 K-I阈值更低，可能会识别出更多暗目标（水体区域）")
        print("      这通常发生在水体类别方差较小的情况下")
    else:
        print("   💡 K-I阈值更高，可能会识别出更少暗目标（水体区域）")
        print("      这通常发生在陆地类别方差较小的情况下")
    
    # === 步骤5: 执行图像分割 ===
    print(f"\n🔄 正在执行图像分割...")
    
    # 基于K-I阈值进行二值分割
    dark_targets_ki = sar_image_for_otsu.lt(ki_threshold_value)    # 暗目标 (SAR < 阈值)
    bright_targets_ki = sar_image_for_otsu.gte(ki_threshold_value) # 亮目标 (SAR >= 阈值)
    
    print(f"   ✅ 暗目标提取完成 (SAR值 < {ki_threshold_value:.4f} dB)")
    print(f"   ✅ 亮目标提取完成 (SAR值 ≥ {ki_threshold_value:.4f} dB)")
    
    # === 步骤6: 创建K-I方法可视化地图 ===
    print(f"\n🗺️  正在创建K-I方法可视化地图...")
    
    Map_KI = geemap.Map()
    Map_KI.centerObject(roi_choose1, 11)  # 居中显示研究区域
    
    # 定义可视化参数
    sar_vis_params = {'min': -25, 'max': 5}                    # SAR影像显示范围
    dark_vis_params = {'palette': ['#0066CC']}                 # 深蓝色 - 暗目标
    bright_vis_params = {'palette': ['#FF6600']}               # 橙色 - 亮目标
    threshold_vis_params = {'palette': ['#FF0000']}            # 红色 - 阈值线
    
    # 添加图层
    Map_KI.addLayer(sar_image_for_otsu, sar_vis_params, 
                    f'📡 SAR Image VV ({targdate})')
    Map_KI.addLayer(dark_targets_ki.selfMask(), dark_vis_params, 
                    f'🌊 Dark Targets - K-I Method (< {ki_threshold_value:.2f} dB)')
    #Map_KI.addLayer(bright_targets_ki.selfMask(), bright_vis_params, 
                    #f'🏔️  Bright Targets - K-I Method (≥ {ki_threshold_value:.2f} dB)')
    
    print("   ✅ K-I分割结果地图创建完成")
    display(Map_KI)
    
    # === 步骤7: 创建方法对比地图 ===
    print(f"\n🔍 正在创建Otsu vs K-I对比地图...")
    
    # 计算两种方法的差异
    methods_difference = dark_targets_otsu.subtract(dark_targets_ki)
    
    Map_Compare = geemap.Map()
    Map_Compare.centerObject(roi_choose1, 11)
    
    # 对比可视化参数
    otsu_color = {'palette': ['#FF3333']}      # 红色 - Otsu独有
    ki_color = {'palette': ['#3333FF']}        # 蓝色 - K-I独有
    overlap_color = {'palette': ['#9933FF']}   # 紫色 - 重叠区域
    diff_vis = {'min': -1, 'max': 1, 'palette': ['#3333FF', '#FFFFFF', '#FF3333']}
    
    # 添加对比图层
    Map_Compare.addLayer(sar_image_for_otsu, sar_vis_params, 
                        f'📡 SAR Background')
    Map_Compare.addLayer(dark_targets_otsu.selfMask(), otsu_color, 
                        f'🔴 Otsu Dark Targets')
    Map_Compare.addLayer(dark_targets_ki.selfMask(), ki_color, 
                        f'🔵 K-I Dark Targets')
    
    # 差异图：红色=仅Otsu，蓝色=仅K-I，白色=无差异
    Map_Compare.addLayer(methods_difference, diff_vis, 
                        '🎨 Difference (Red=Otsu only, Blue=K-I only)')
    
    print("   ✅ 方法对比地图创建完成")
    display(Map_Compare)
    
    # === 步骤8: 定量统计分析 ===
    print(f"\n📊 === 定量统计分析 ===")
    
    # 计算各方法识别的暗目标像元数量
    print("🔢 正在计算像元统计...")
    
    otsu_pixel_stats = dark_targets_otsu.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi_choose1,
        scale=30,
        maxPixels=1e10
    )
    
    ki_pixel_stats = dark_targets_ki.reduceRegion(
        reducer=ee.Reducer.sum(), 
        geometry=roi_choose1,
        scale=30,
        maxPixels=1e10
    )
    
    # 获取统计结果
    otsu_pixel_count = otsu_pixel_stats.getInfo()['VV']
    ki_pixel_count = ki_pixel_stats.getInfo()['VV']
    
    # 计算面积（假设像元大小为30m×30m=900m²）
    pixel_area = 900  # 平方米
    otsu_area_km2 = (otsu_pixel_count * pixel_area) / 1e6
    ki_area_km2 = (ki_pixel_count * pixel_area) / 1e6
    
    # 显示统计结果
    print(f"\n📋 暗目标识别统计:")
    print(f"   🎯 Otsu方法:")
    print(f"      - 像元数量: {otsu_pixel_count:,} 个")
    print(f"      - 覆盖面积: {otsu_area_km2:.3f} km²")
    print(f"   🎯 K-I方法:")
    print(f"      - 像元数量: {ki_pixel_count:,} 个")
    print(f"      - 覆盖面积: {ki_area_km2:.3f} km²")
    
    # 计算相对差异
    if otsu_pixel_count > 0:
        relative_diff_percent = ((ki_pixel_count - otsu_pixel_count) / otsu_pixel_count) * 100
        area_diff_km2 = ki_area_km2 - otsu_area_km2
        
        print(f"\n📈 差异分析:")
        print(f"   📏 像元数差异: {ki_pixel_count - otsu_pixel_count:,} 个")
        print(f"   📏 面积差异: {area_diff_km2:+.3f} km²")
        print(f"   📊 相对差异: {relative_diff_percent:+.2f}%")
        
        if abs(relative_diff_percent) < 5:
            print(f"   💡 结论: 两种方法结果非常相似")
        elif relative_diff_percent > 0:
            print(f"   💡 结论: K-I方法识别出更多暗目标")
        else:
            print(f"   💡 结论: Otsu方法识别出更多暗目标")
    
    print(f"\n✅ Kittler-Illingworth分割方法分析完成!")

else:
    print("❌ 错误：未找到目标日期的SAR影像，无法执行K-I方法")


🔬 === Kittler-Illingworth 最小误差分割方法 ===
📅 处理日期: 2024-10-31
🛰️  使用影像数量: 1 景

📡 使用相同的预处理SAR影像...
   - 波段: VV极化
   - 滤波核半径: 2 像素
   - 研究区域: 已裁剪到AOI

📊 正在计算SAR影像的直方图...
   - 这是K-I方法的基础数据
   - 直方图将用于估计两个高斯分布的参数

🧮 正在应用Kittler-Illingworth算法...
   - 假设: SAR图像由暗目标(水体)和亮目标(陆地)两个高斯分布组成
   - 目标: 找到使贝叶斯分类误差最小的阈值
✅ K-I最小误差阈值计算完成: -16.5000 dB

📈 === 阈值对比分析 ===
   🎯 Otsu阈值 (最大类间方差):     -13.5002 dB
   🎯 K-I阈值 (最小分类误差):      -16.5000 dB
   📏 阈值差异:                    -2.9999 dB
   💡 K-I阈值更低，可能会识别出更多暗目标（水体区域）
      这通常发生在水体类别方差较小的情况下

🔄 正在执行图像分割...
   ✅ 暗目标提取完成 (SAR值 < -16.5000 dB)
   ✅ 亮目标提取完成 (SAR值 ≥ -16.5000 dB)

🗺️  正在创建K-I方法可视化地图...
   ✅ K-I分割结果地图创建完成


Map(center=[39.39610997738956, -0.377311304937758], controls=(WidgetControl(options=['position', 'transparent_…


🔍 正在创建Otsu vs K-I对比地图...
   ✅ 方法对比地图创建完成


Map(center=[39.39610997738956, -0.377311304937758], controls=(WidgetControl(options=['position', 'transparent_…


📊 === 定量统计分析 ===
🔢 正在计算像元统计...

📋 暗目标识别统计:
   🎯 Otsu方法:
      - 像元数量: 50,053.91764705875 个
      - 覆盖面积: 45.049 km²
   🎯 K-I方法:
      - 像元数量: 43,439.23921568625 个
      - 覆盖面积: 39.095 km²

📈 差异分析:
   📏 像元数差异: -6,614.6784313725 个
   📏 面积差异: -5.953 km²
   📊 相对差异: -13.22%
   💡 结论: Otsu方法识别出更多暗目标

✅ Kittler-Illingworth分割方法分析完成!


## 5.3 采用K-Means方法监督

In [ ]:
# --- K-Means方法已注释掉 ---
# sar_image_dual = s1_target_collection.select(['VV','VH']).mosaic().clip(roi_choose1)

# # --- 1. 特征工程：计算比值和和值并合并波段 ---
# print("正在计算 VV/VH 比值和 VV+VH 和值特征...")
    
# # 提取VV和VH波段，方便进行计算
# vv_band = sar_image_dual.select('VV')
# vh_band = sar_image_dual.select('VH')
    
# # 计算比值 (Ratio): VV / VH
# # 比值可以放大水体和部分植被的差异
# ratio_band = vv_band.divide(vh_band).rename('ratio')
    
# # 计算和值 (Sum): VV + VH
# # 和值可以综合反映总体的后向散射能量
# sum_band = vv_band.add(vh_band).rename('sum')
    
# # 将四个特征波段合并成一个多维特征影像
# feature_image = ee.Image.cat([vv_band, vh_band, ratio_band, sum_band])
    
# print("4维特征影像创建完成: [VV, VH, ratio, sum]")

# # --- 2. K-Means聚类 ---
# num_clusters = 5 # 对于4维特征，聚类数为5
# num_samples = 5000
# print(f"正在从特征影像中随机采样 {num_samples} 个点用于训练...")
# training_samples = feature_image.sample(
#         region=roi_choose1,
#         scale=10,
#         numPixels=num_samples,
#         seed=0 # 添加一个随机种子确保每次运行结果一致
#     )

# # 初始化并训练K-Means聚类器
# clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(training_samples)

# # 使用训练好的聚类器对整个特征影像分类
# print("正在对整个研究区进行分类...")
# clustered_image = feature_image.cluster(clusterer)

# # --- 3.自动识别水体类别 ---
# # 基于水体聚类VV平均值最低的原则，识别水体类
# print("正在识别水体类别")

# # 创建一个图像，波段1是每个像元VV值，波段2是聚类ID，这样可以统计哪个id对应的聚类平均VV最低
# sort_image = feature_image.select('VV').addBands(clustered_image)

# # 根据聚类id统计各类别VV均值
# # 得到的是一个列表:{cluster:1,mean:-19.7}这样的
# means_dict = sort_image.reduceRegion(
#     reducer = ee.Reducer.mean().group(groupField=1 , groupName='cluster'),
#     geometry = roi_choose1,
#     scale = 30,
#     maxPixels = 1e9
# )

# # 在客户端寻找mean值最小的类别
# groups = ee.List(means_dict.get('groups')).getInfo()
# min_mean = float('inf') #初始值设置为无穷大
# water_cluster_id = -1
# # 冒泡排序
# for group in groups:
#     if group['mean'] < min_mean:
#         min_mean = group['mean']
#         water_cluster_id = group['cluster']

# if water_cluster_id != -1:
#     print(f"识别完成！水体类别ID为: {water_cluster_id} (平均VV值为: {min_mean:.4f})")
#     initial_water_kmeans = clustered_image.eq(water_cluster_id)
# else:
#     print("错误：未能成功识别水体类别。")
#     # 创建一个空影像以避免后续代码出错
#     initial_water_kmeans = ee.Image(0).selfMask()

#  # --- 5. 移除永久性水体 ---
# jrc_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
# occurrence_unmasked = jrc_water.select('occurrence').unmask(0)
# permanent_water_mask = occurrence_unmasked.gt(90)
    
# flood_map_kmeans = initial_water_kmeans.And(permanent_water_mask.Not())
# print("已从K-Means结果中移除永久性水体，生成最终洪水图。")

# # --- 6. 可视化结果 ---
# print("正在准备可视化地图...")
# Map_KMeans_New = geemap.Map()
# Map_KMeans_New.centerObject(roi_choose1, 11)

# sar_vis_params = {'min': -25, 'max': 0}
# water_vis_params = {'palette': ['#0000FF']}
# cluster_palette = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF']
# cluster_vis_params = {'min': 0, 'max': num_clusters - 1, 'palette': cluster_palette[:num_clusters]}

# Map_KMeans_New.addLayer(sar_image_dual.select('VV'), sar_vis_params, f'SAR Image VV ({targdate})')
# Map_KMeans_New.addLayer(clustered_image, cluster_vis_params, 'K-Means Clusters (Ratio/Sum)')
# Map_KMeans_New.addLayer(initial_water_kmeans.selfMask(), {'palette': ['#FFFF00']}, 'Initial Water (K-Means)')
# Map_KMeans_New.addLayer(flood_map_kmeans.selfMask(), water_vis_params, 'Final Flood (K-Means Ratio/Sum - JRC)')

# Map_KMeans_New

print("K-Means方法已注释掉，如需使用请取消注释。")

正在计算 VV/VH 比值和 VV+VH 和值特征...
4维特征影像创建完成: [VV, VH, ratio, sum]
正在从特征影像中随机采样 5000 个点用于训练...
正在对整个研究区进行分类...
正在识别水体类别
识别完成！水体类别ID为: 2 (平均VV值为: -20.1734)
已从K-Means结果中移除永久性水体，生成最终洪水图。
正在准备可视化地图...


Map(center=[39.41956382721997, -0.431725495619315], controls=(WidgetControl(options=['position', 'transparent_…

## 5.3 Z分数固定阈值分割洪水



In [ ]:
# --- Z-Score方法已注释掉 ---
# z_score_image = z.map(lambda image: image.clip(roi_choose1)).mosaic().select('VV')

# Z_THRESHOLD = -2.5

# print(f"正在应用固定的Z-Score阈值: Z < {Z_THRESHOLD}")
    
# # 应用阈值：Z-Score值低于阈值的像元被分类为水体 (值为1)
# initial_water_zscore = z_score_image.lt(Z_THRESHOLD)

# # --- 3. 移除永久性水体 ---
# # (此逻辑与其他对照方法完全相同)
# jrc_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
# occurrence_unmasked = jrc_water.select('occurrence').unmask(0)
# permanent_water_mask = occurrence_unmasked.gt(90)
    
# flood_map_zscore = initial_water_zscore.And(permanent_water_mask.Not())
# print("已从Z-Score阈值结果中移除永久性水体，生成最终洪水图。")

# # --- 4. 可视化结果 ---
# print("正在准备可视化地图...")
# Map_ZScore_Threshold = geemap.Map()
# Map_ZScore_Threshold.centerObject(roi_choose1, 11)

# # Z-Score影像的可视化参数 (使用你论文中统一的配色)
# zpalette = ['#b2182b','#ef8a62','#fddbc7','#f7f7f7','#d1e5f0','#67a9cf','#2166ac']
# z_score_vis_params = {'min': -5, 'max': 5, 'palette': zpalette}
    
# # 水体的可视化参数
# water_vis_params = {'palette': ['#0000FF']}

# # 添加图层以便对比
# Map_ZScore_Threshold.addLayer(z_score_image, z_score_vis_params, f'Z-Score Image ({targdate})')
# Map_ZScore_Threshold.addLayer(initial_water_zscore.selfMask(), {'palette': ['#FFFF00']}, f'Initial Water (Z < {Z_THRESHOLD})')
# Map_ZScore_Threshold.addLayer(flood_map_zscore.selfMask(), water_vis_params, 'Final Flood (Z-Score Threshold - JRC)')
    
# Map_ZScore_Threshold

print("Z-Score方法已注释掉，如需使用请取消注释。")

正在应用固定的Z-Score阈值: Z < -2.5
已从Z-Score阈值结果中移除永久性水体，生成最终洪水图。
正在准备可视化地图...


Map(center=[39.41956382721997, -0.431725495619315], controls=(WidgetControl(options=['position', 'transparent_…

# 6.导出对照方法结果


In [ ]:
# --- 1. 定义通用的导出参数 ---

# 定义一个统一的文件夹名，所有结果都会保存在这里
# 使用目标日期命名，方便归档
folder_name = f"Image_Segmentation_Results_{targdate}"

# 定义导出分辨率 (米)
export_scale = 10 

# 定义导出区域
export_region = roi_choose1 # 使用.geometry()确保传递的是纯几何对象

print(f"所有文件将被导出到Google Drive的 '{folder_name}' 文件夹中。")
print("请稍后前往GEE Code Editor的 'Tasks' 标签页手动运行所有任务。")

# --- 2. 创建一个待导出影像的字典 ---
# 这样做代码更整洁，方便管理
# 键(key)是文件名，值(value)是影像对象
image_exports = {
    'Dark_Targets_Otsu': 'dark_targets_otsu',
    'Bright_Targets_Otsu': 'bright_targets_otsu'
    # 'Flood_Map_KMeans_JRC': 'flood_map_kmeans',  # 已注释
    # 'Flood_Map_ZScore_JRC': 'flood_map_zscore'   # 已注释
}

# --- 3. 循环提交导出任务 ---

for filename, image_var_name in image_exports.items():
    
    # 检查影像变量是否存在，以防某个方法之前运行失败
    if image_var_name in globals():
        
        # 从全局变量中获取影像对象
        image_to_export = globals()[image_var_name]
        
        # 准备任务
        task = ee.batch.Export.image.toDrive(
            # 将二值影像转为Byte类型，可以极大减小文件体积
            image=image_to_export.toByte().clip(export_region), 
            description=f'Export_{filename}', # GEE Tasks面板中显示的任务名
            folder=folder_name,
            fileNamePrefix=filename, # 最终的文件名
            region=export_region,
            scale=export_scale,
            fileFormat='GeoTIFF',
            maxPixels=1e13 
        )
        
        # 启动任务
        task.start()
        
        print(f"  > 成功提交影像 '{filename}.tif' 的导出任务。")
    
    else:
        print(f"  > 警告：未找到名为 '{image_var_name}' 的影像变量，跳过导出。")

所有文件将被导出到Google Drive的 'Flood_Comparison_Results_2024-10-31' 文件夹中。
请稍后前往GEE Code Editor的 'Tasks' 标签页手动运行所有任务。
  > 成功提交影像 'Flood_Map_Otsu_JRC.tif' 的导出任务。
  > 成功提交影像 'Flood_Map_KMeans_JRC.tif' 的导出任务。
  > 成功提交影像 'Flood_Map_ZScore_JRC.tif' 的导出任务。
